In [1]:
from ultralytics import YOLO
import mediapipe as mp
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import numpy as np
import os
import random
import platform
import yaml
from tqdm import tqdm
import gc
from torch.cuda import empty_cache

if platform.system() == "Darwin":
    print("your system is mac os")
    device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
else:
    print("your system is cuda")
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

your system is cuda


In [25]:
data_dir = '/home/pepsi/Downloads/road_dataset_yolo_real_final'
model = 'yolov8m.pt'

classes = ['solid', 'center', 'dotted']
nc = len(classes)
yaml_file = "data.yaml"

In [26]:
data = {
    "train": data_dir + "/train/",
    "val": data_dir + "/val/",
    "test": data_dir + "/test/",
    "nc": nc,
    "names": classes
}

In [27]:
os.makedirs(data_dir, exist_ok=True)

with open(data_dir + "/" + yaml_file, "wt") as ft:
    yaml.dump(data, ft)

In [28]:
with open(data_dir + "/" + yaml_file, "rt") as fr:
    d = yaml.safe_load(fr)
    print(type(d))
    print(d)

<class 'dict'>
{'names': ['solid', 'center', 'dotted'], 'nc': 3, 'test': '/home/pepsi/Downloads/road_dataset_yolo_real_final/test/', 'train': '/home/pepsi/Downloads/road_dataset_yolo_real_final/train/', 'val': '/home/pepsi/Downloads/road_dataset_yolo_real_final/val/'}


In [29]:
training_epochs = 200
patience = 30
batch = 8
imgsz = 640
name = "yolov8_epoch200"

In [30]:
yolo_model = YOLO("yolov8m-seg.pt").to(device)

yolo_model.train(data=data_dir + "/" + yaml_file, epochs=training_epochs, patience=patience, batch=batch, imgsz=imgsz, name=name)

New https://pypi.org/project/ultralytics/8.3.136 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/home/pepsi/Downloads/road_dataset_yolo_real_final/data.yaml, epochs=200, time=None, patience=30, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=yolov8_epoch2006, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

100%|██████████| 5.35M/5.35M [00:00<00:00, 92.7MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 755.2±402.1 MB/s, size: 43.4 KB)


train: Scanning /home/pepsi/Downloads/road_dataset_yolo_real_final/train/labels... 1255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1255/1255 [00:01<00:00, 1008.36it/s]

train: New cache created: /home/pepsi/Downloads/road_dataset_yolo_real_final/train/labels.cache


...

                   all        314        656      0.988      0.985       0.99      0.949      0.991      0.984      0.993      0.838



200 epochs completed in 3.782 hours.
Optimizer stripped from runs/segment/yolov8_epoch2006/weights/last.pt, 54.8MB
Optimizer stripped from runs/segment/yolov8_epoch2006/weights/best.pt, 54.8MB

Validating runs/segment/yolov8_epoch2006/weights/best.pt...
Ultralytics 8.3.118 🚀 Python-3.12.3 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 5920MiB)
YOLOv8m-seg summary (fused): 105 layers, 27,224,121 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:08<00:00,  2.49it/s]


                   all        314        656      0.984      0.985      0.989       0.95      0.986      0.986      0.993      0.843
                 solid        270        271      0.996      0.996      0.995       0.97      0.996      0.996      0.995      0.915
                center        248        258      0.988      0.994      0.994      0.975      0.977      0.982      0.993      0.878
                dotted        100        127      0.968      0.965      0.979      0.904      0.984       0.98       0.99      0.737
Speed: 0.7ms preprocess, 10.4ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/segment/yolov8_epoch2006


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x774b26d6c7a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04

In [31]:
classes = ['solid','center', 'dotted']

yolo_best_model_path = "/home/pepsi/dev_ws/ros-repo-5/admin/src/ai_server/ai_train/runs/segment/yolov8_epoch2006/weights/best.pt"
yolo_best_model = YOLO(yolo_best_model_path).to(device)

metrics = yolo_best_model.val()

YOLOv8m-seg summary (fused): 105 layers, 27,224,121 parameters, 0 gradients, 110.0 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1133.5±654.3 MB/s, size: 42.3 KB)


val: Scanning /home/pepsi/Downloads/road_dataset_yolo_real_final/val/labels.cache... 314 images, 0 backgrounds, 0 corrupt: 100%|██████████| 314/314 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:13<00:00,  1.52it/s]


                   all        314        656      0.984      0.985      0.989      0.949      0.987      0.987      0.993      0.841
                 solid        270        271      0.996      0.996      0.995       0.97      0.996      0.996      0.995      0.913
                center        248        258      0.988      0.994      0.994      0.974      0.981      0.986      0.993      0.871
                dotted        100        127      0.968      0.964      0.978      0.902      0.984       0.98       0.99      0.738
Speed: 2.2ms preprocess, 18.4ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/segment/val2
